## Welcome to Lab 3 for Week 1 Day 4

Today we're going to build something with immediate value!

In the folder `me` I've put a single file `linkedin.pdf` - it's a PDF download of my LinkedIn profile.

Please replace it with yours!

I've also made a file called `summary.txt`

We're not going to use Tools just yet - we're going to add the tool tomorrow.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/tools.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Looking up packages</h2>
            <span style="color:#00bfff;">In this lab, we're going to use the wonderful Gradio package for building quick UIs, 
            and we're also going to use the popular PyPDF PDF reader. You can get guides to these packages by asking 
            ChatGPT or Claude, and you find all open-source packages on the repository <a href="https://pypi.org">https://pypi.org</a>.
            </span>
        </td>
    </tr>
</table>

In [15]:
# If you don't know what any of these packages do - you can always ask ChatGPT for a guide!
from dotenv import load_dotenv
from openai import OpenAI
from PyPDF2 import PdfReader
import gradio as gr


In [2]:
load_dotenv(override=True)
openai = OpenAI()

In [5]:
reader = PdfReader("me/linkedin.pdf")
linkedin = ""
for pages in reader.pages:
    text = pages.extract_text()
    if text:
        linkedin += text

print(linkedin)

   
Contactar
Madrid, Spain
635954292  (Mobile)
aitorbermeruiz@gmail.com
www.linkedin.com/in/aitor-bermejo-
ruiz-991722164  (LinkedIn)
drive.google.com/drive/
folders/1aO__Dj_egryLkfrylhRtlOCXhG0r2_5l
(Personal)
Aptitudes principales
Inglés
.NET Core
Servicios web y API REST
Languages
Inglés  (Full Professional)
Español  (Native or Bilingual)
Certifications
Microsoft Certified: Azure Developer
Associate
Android Studio completo (Java)
Microsoft Certified: Azure
FundamentalsAitor Bermejo Ruiz
Cloud & API Solution Developer
Madrid y alrededores
Extracto
Soy un desarrollador apasionado especializado en la creación de
aplicaciones multiplataforma y soluciones en la nube con más de
dos años de experiencia como Cloud & API Solution Developer.
Mi trayectoria profesional se centra en el desarrollo de Web APIs,
Azure Functions y Logic Apps en Microsoft Azure, así como
en la gestión completa del ciclo de vida de las APIs mediante
herramientas como Azure API Management destacando el esfuerzo
de la

In [6]:
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

print(summary)


Mi nombre es Aitor Bermejo, de España y tengo 23 años. Soy un desarrollador de software con mas de 3 años carrera especializado en .NET, Azure y APIRest. He estado gran parte de mi carrera trabajando como APIDesigner pero ahora me dedico tambien a parte del desarrollo de estas propias APIs. He definido a lo largo de mis años en la empresa una metodología APIFirst uncluyendo un modelo novedisi como lo es ApiOps.

Soy un chico joven, social y amable, pero aunque no lo parezca mi sueño es vivir en el monte alejano de la poblacion(pero no demasiado), vivir en un chalet con gimnasio, huerto y gallinas e incluso vacas. Y poder dedicarme a beber cafe mientras miro los picos de europa. Pero me encanta el sector privado y la motivacion que te da.


In [7]:
name = "Aitor Bermejo"

In [8]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."

In [18]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content


gr.ChatInterface(chat, type= )

In [19]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7866
* To create a public link, set `share=True` in `launch()`.


## Special note for people not using OpenAI

Some providers, like Groq, might give an error when you send your second message in the chat.

This is because Gradio shoves some extra fields into the history object. OpenAI doesn't mind; but some other models complain.

If this happens, the solution is to add this first line to the chat() function above. It cleans up the history variable:

```python
history = [{"role": h["role"], "content": h["content"]} for h in history]
```

You may need to add this in other chat() callback functions in the future, too.

In [ ]:
gr.ChatInterface(chat, type="messages").launch()

## A lot is about to happen...

1. Be able to ask an LLM to evaluate an answer
2. Be able to rerun if the answer fails evaluation
3. Put this together into 1 workflow

All without any Agentic framework!

In [20]:
# Create a Pydantic model for the Evaluation

from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str


In [21]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [22]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += "Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [23]:
import os
gemini = OpenAI(
    api_key=os.getenv("GOOGLE_API_KEY"), 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

In [28]:
def evaluate(reply, message, history) -> Evaluation:

    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = gemini.beta.chat.completions.parse(model="gemini-2.0-flash", messages=messages, response_format=Evaluation)
    print(f"Messafe: {message}\nResponse: {response.choices[0].message.parsed}")
    return response.choices[0].message.parsed
    

In [29]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "do you hold a patent?"}]
response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
reply = response.choices[0].message.content

In [32]:
reply
messages[:1]

[{'role': 'system',
  'content': "You are acting as Aitor Bermejo. You are answering questions on Aitor Bermejo's website, particularly questions related to Aitor Bermejo's career, background, skills and experience. Your responsibility is to represent Aitor Bermejo for interactions on the website as faithfully as possible. You are given a summary of Aitor Bermejo's background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.\n\n## Summary:\nMi nombre es Aitor Bermejo, de España y tengo 23 años. Soy un desarrollador de software con mas de 3 años carrera especializado en .NET, Azure y APIRest. He estado gran parte de mi carrera trabajando como APIDesigner pero ahora me dedico tambien a parte del desarrollo de estas propias APIs. He definido a lo largo de mis años en la empresa una metodología APIFirst uncluyendo un modelo novedi

In [33]:
evaluate(reply, "do you hold a patent?", messages[:1])

Messafe: do you hold a patent?
Response: is_acceptable=True feedback='The response is very good. It is accurate and professional, and it is very much in line with the persona and instructions provided. The answer also invites further questions, which is helpful in keeping the conversation going.'


Evaluation(is_acceptable=True, feedback='The response is very good. It is accurate and professional, and it is very much in line with the persona and instructions provided. The answer also invites further questions, which is helpful in keeping the conversation going.')

In [34]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + "\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [ ]:
def chat(message, history):
    if "patent" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    reply =response.choices[0].message.content
    print("Reply: ", reply)

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

In [ ]:
from dotenv import load_dotenv
from openai import OpenAI
from PyPDF2 import PdfReader
import gradio as gr
from pydantic import BaseModel
import os

In [48]:
# CREAR TODO DE NUEVO CON COMENTARIOS!!!!
load_dotenv(override=True)
openai = OpenAI()

# 1. CREAMOS CLASE
class Evaluation(BaseModel):
    isAcceptable: bool
    feedback: str

# LEEMOS PDF
reader = PdfReader("me/linkedin.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
       linkedin += text

# LEEMOS SUMARY
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

# CREAMOS FUNCIONES PARA CHAT.
gemini = OpenAI(api_key=os.getenv("GOOGLE_API_KEY"), base_url="https://generativelanguage.googleapis.com/v1beta/openai/")

name = "Aitor Bermejo"

system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
    particularly questions related to {name}'s career, background, skills and experience. \
    Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
    You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
    Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
    If you don't know the answer, say so."                      

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."

def evaluate(message, reply, history) -> Evaluation:
    evaluate_user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n" \
        f"Here's the latest message from the User: \n\n{message}\n\n" \
        f"Here's the latest response from the Agent: \n\n{reply}\n\n" \
        "Please evaluate the response, replying with whether it is acceptable and your feedback." 

    evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
        You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
        The Agent is playing the role of {name} and is representing {name} on their website. \
        The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
        The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

    evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n" \
        f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluate_user_prompt}]
    response = gemini.beta.chat.completions.parse(model="gemini-2.0-flash", messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

def rerun(reply, message, history, feedback):
        updated_system_prompt = system_prompt + "\n\n ##Previous answer rejected \n\n J¡You just try to reply, but the quality rejected your response" 
        updated_system_prompt += f"\n ## Your attempted answer: \n{reply}"
        updated_system_prompt += f"\n \n Reason for rejection: \n{feedback}"
        messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]                        
        response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
        return response.choices[0].message.content
        

def chat(message, history):
    if "patent" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    reply =response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.isAcceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7872
* To create a public link, set `share=True` in `launch()`.
